# Setup

In [1]:
import pandas as pd
import numpy as np

from eval_funcs import (
        perplexity_for_corpora,
        wasserstein_distance_embeddings,
        classify_real_vs_synth,
        compute_stat_properties
    )

# Import Real Dataset

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("gowrishankarp/newspaper-text-summarization-cnn-dailymail")

print("Path to dataset files:", path)

Path to dataset files: C:\Users\loren\.cache\kagglehub\datasets\gowrishankarp\newspaper-text-summarization-cnn-dailymail\versions\2


In [3]:
cnn_train = pd.read_csv(r'C:\Users\loren\.cache\kagglehub\datasets\gowrishankarp\newspaper-text-summarization-cnn-dailymail\versions\2\cnn_dailymail\train.csv')

In [4]:
cnn_train.head(2)

,id,article,highlights
0,0001d1afc246a7964130f43ae940af6bc6c57f01,By . Associated Press . PUBLISHED: . 14:11 EST...,"Bishop John Folda, of North Dakota, is taking ..."
1,0002095e55fcbd3a2f366d9bf92a95433dc305ef,(CNN) -- Ralph Mata was an internal affairs li...,Criminal complaint: Cop used his role to help ...


# Import TinyLlama Synthetic Dataset

In [5]:
tinyllama_data = pd.read_csv('../tinyllama_outputs.csv').drop('Unnamed: 0', axis=1)

In [6]:
tinyllama_data.head(2)

,uuid,topic,generated_article,elapsed_time
0,00b75438-5d31-4f38-add3-f873c44fdb65,Politics - Elections,"Fearing a catastrophic defeat at the polls, US...",5.364199
1,3a1296a5-83f0-4945-a105-74a1117758b6,World - Middle East,Bombarded by violent attacks in mid-East: CNN ...,4.014536


In [7]:
tinyllama_data['generated_article'].shape

(1000,)

In [8]:
tinyllama_data['generated_article'].str.split(' ').transform(lambda x: len(x)).max()

2396

# Run Eval Functions

### Running Statistical Properties Metrics on TinyLlama Data

In [9]:
stats = compute_stat_properties(tinyllama_data['generated_article'], max_length=4096)

In [10]:
stats

{'avg_len_tokens': 825.7349853515625,
 'std_len_tokens': 352.1277160644531,
 'avg_len_chars': 4039.864,
 'ttr': 0.031341774298049616,
 'hapax_ratio': 0.26866306027820713}

### Running Perplexity Scores

In [ ]:
ppl = perplexity_for_corpora(cnn_train['article'].sample(1000, random_state=42), tinyllama_data['generated_article'], batch_size=8, max_length=2048)

[perplexity] device=cuda batch_size=8 max_length(requested)=2048 max_length(effective)=2048
[perplexity] num_docs: real=1000 synthetic=1000 total_batches=250


c:\Users\loren\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\opt\modeling_opt.py:341: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


[perplexity] progress 5/250 | elapsed=5.3s | avg/batch=1.05s | ETA~258.3s
[perplexity] progress 10/250 | elapsed=10.6s | avg/batch=1.06s | ETA~253.8s


In [ ]:
ppl

### Running Wasserstein Distance

In [ ]:
wd = wasserstein_distance_embeddings(cnn_train['article'].sample(1000, random_state=42), tinyllama_data['generated_article'], n_projections=128)

In [ ]:
wd

### Runnning Classification (Real vs. Synthetic)

In [ ]:
clf_res = classify_real_vs_synth(cnn_train['article'].sample(1000, random_state=42), tinyllama_data['generated_article'], cv=5)

In [ ]:
clf_res

# Visualize?